<a href="https://colab.research.google.com/github/Campfireman0314/Campfireman0314/blob/main/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install hopsworks

from huggingface_hub import notebook_login

notebook_login()


Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
import hopsworks
project = hopsworks.login()

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated

Paste it here: ··········
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/4258


login hopsworks and huggingface

prepare

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "zh-CN", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "zh-CN", split="test", use_auth_token=True)

print(common_voice)


import mandarin dataset

In [ ]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])


remove data which are not used

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")


Downloading:   0%|          | 0.00/185k [00:00<?, ?B/s]

import WhisperFeatureExtractor with medium data size

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Chinese", task="transcribe")


Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/494k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

import WhisperTokenizer

In [ ]:
input_str = common_voice["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")


Input:                 性喜温暖润湿气候且耐寒。
Decoded w/ special:    <|startoftranscript|><|zh|><|transcribe|><|notimestamps|>性喜温暖润湿气候且耐寒。<|endoftext|>
Decoded w/out special: 性喜温暖润湿气候且耐寒。
Are equal:             True


a test for WhisperTokenizer

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Chinese", task="transcribe")


combine whisperTokenizer and WhisperFeatureExtractor to a function called WhisperProcessor

In [ ]:
print(common_voice["train"][0])


{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/892af1724096c1e9ed1d53df5f8c14697ce16fc1e756f4c7b56c93aa20bc0625/common_voice_zh-CN_33211332.mp3', 'array': array([-2.0447670e-14, -2.1661141e-12, -2.8800445e-12, ...,
        1.2203335e-05,  3.2381283e-06, -2.7760734e-07], dtype=float32), 'sampling_rate': 48000}, 'sentence': '性喜温暖润湿气候且耐寒。'}


print data[0]

In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))


In [ ]:
print(common_voice["train"][0])


{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/892af1724096c1e9ed1d53df5f8c14697ce16fc1e756f4c7b56c93aa20bc0625/common_voice_zh-CN_33211332.mp3', 'array': array([-9.3524797e-13, -1.4627063e-12,  1.9229006e-12, ...,
       -5.5482551e-06,  2.6667838e-05,  1.3390082e-05], dtype=float32), 'sampling_rate': 16000}, 'sentence': '性喜温暖润湿气候且耐寒。'}


sampling with 16kHZ

In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch


In [ ]:
print(common_voice["train"][0])


{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/892af1724096c1e9ed1d53df5f8c14697ce16fc1e756f4c7b56c93aa20bc0625/common_voice_zh-CN_33211332.mp3', 'array': array([-9.3524797e-13, -1.4627063e-12,  1.9229006e-12, ...,
       -5.5482551e-06,  2.6667838e-05,  1.3390082e-05], dtype=float32), 'sampling_rate': 16000}, 'sentence': '性喜温暖润湿气候且耐寒。'}


prepare the data with id and add labels

In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=1)


  0%|          | 0/39637 [00:00<?, ?ex/s]